# Integrators and sampling

In [56]:
# Preliminary imports
from simtk import openmm, unit
import numpy as np

We'll use a Lennard-Jones system again, this time imported from [`openmmtools`](http://openmmtools.readthedocs.io):

In [57]:
# Create a Lennard-Jones fluid
nparticles = 512
reduced_density = 0.8
mass = 39.9 * unit.amu
charge = 0.0 * unit.elementary_charge
sigma = 3.4 * unit.angstroms
epsilon = 0.238 * unit.kilocalories_per_mole
from openmmtools import testsystems
testsystem = testsystems.LennardJonesFluid(nparticles=nparticles, mass=mass, reduced_density=reduced_density, sigma=sigma, epsilon=epsilon, switch_width=0.5*sigma, cutoff=3*sigma)
system, positions = testsystem.system, testsystem.positions

## Built-in OpenMM integrators

OpenMM provides a number of built-in [`Integrators`](http://docs.openmm.org/7.1.0/userguide/application.html#integrators) that may be useful for your intended application, but also provides a very flexible way to efficiently define new integrators that can still be executed very efficiently on the GPU.

All OpenMM integrators are derived from the [`Integrator`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Integrator.html#simtk.openmm.openmm.Integrator) base class that provides some baseline functionality such that all integrators work the same way. We first have to bind an integrator to a `Context`. 

In [63]:
# Create a new integrator since the previously-created integrator was irrevocably bound to the previous Context
temperature = 0.9 * (epsilon / (unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA))
timescale = 0.09354 * unit.sqrt((mass * sigma**2) / epsilon) # relevant timescale for Lennard-Jones vibration
collision_rate = 1.0 / timescale
timestep = 0.1 * timescale
integrator = openmm.LangevinIntegrator(temperature, collision_rate, timestep)
# Create a Context for this integrator
context = openmm.Context(system, integrator)
# Set the positions
context.setPositions(positions)
# Minimize the potential energy
openmm.LocalEnergyMinimizer.minimize(context)

The initial velocities are, by default, zero, but we can select initial velocities from the Maxwell distribution with [`context.setVelocitiesToTemperature()`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Context.html#simtk.openmm.openmm.Context.setVelocitiesToTemperature):

In [64]:
# Set velocities from Maxwell-Boltzmann distribution
context.setVelocitiesToTemperature(temperature)

To integrate a trajectory, we call [`integrator.step`](http://docs.openmm.org/7.1.0/api-python/generated/simtk.openmm.openmm.Integrator.html#simtk.openmm.openmm.Integrator.step):

In [65]:
# Integrate some dynamics
nsteps = 100 # number of integrator steps
integrator.step(nsteps)

We can write a little loop to report some information every few timesteps:

In [66]:
# Run a few iterations of a few steps each, reporting potential energy
for iteration in range(10):
    integrator.step(100)
    state = context.getState(getEnergy=True)
    print('%8.3f ps : potential %12.6f kJ/mol' % (state.getTime() / unit.picoseconds, state.getPotentialEnergy() / unit.kilojoules_per_mole))

   4.026 ps : potential -2850.972660 kJ/mol
   6.039 ps : potential -2858.054691 kJ/mol
   8.053 ps : potential -2841.517826 kJ/mol
  10.066 ps : potential -2851.142338 kJ/mol
  12.079 ps : potential -2852.352299 kJ/mol
  14.092 ps : potential -2852.897709 kJ/mol
  16.105 ps : potential -2864.873295 kJ/mol
  18.118 ps : potential -2897.962406 kJ/mol
  20.132 ps : potential -2893.789066 kJ/mol
  22.145 ps : potential -2849.431156 kJ/mol
